# Preprocessing

In [1]:
import pandas as pd
import numpy as np
import sys

### Step 1
Write a function that processes a single .txt file. It must:
* drop rows that do not contain a unique school identifier.
* drop rows that correspond to elementary/middle school education. We are focusing on high school data.

In [2]:
def read_text_file(textfilepath):
    """
    Input: textfilepath, a path to the text file to be generated to a datafrmae
    Output: Pandas DataFrame corresponding to input text file
    """
    df = pd.read_csv(textfilepath, sep="\t", encoding="ISO-8859-1")
    return df 

In [3]:
df1 = read_text_file("./txt_files/absenteeism_txt_files/2016-17_ChronAbsenteeism.txt")
df2 = read_text_file("./txt_files/absenteeism_txt_files/2017-18_ChronAbsenteeism.txt")

In [4]:
def drop_rows(df):
    """
    Input: Pandas DataFrame
    Output: Pandas DataFrame with (a) rows with no school code and (b) rows corresponding to elementary/
            middle school education removed
    """
    rows_to_drop = []
    #drop rows that do not have unique school code
    for i, code in enumerate(df["SchoolCode"]):
        if pd.isnull(code) or code == 0:
            rows_to_drop.append(i)
    #drop rows that correspond to elementary and middle school data
    for i, reporting_category in enumerate(df["ReportingCategory"]):
        if reporting_category in ["GRKN", "GRK", "GR13", "GR46", "GR78", "GRK8", "GR912", "GRUG"]:
            rows_to_drop.append(i)
    df = df.drop(rows_to_drop,axis=0)
    return df

In [5]:
df1 = drop_rows(df1)
df2 = drop_rows(df2)

### Step 2
We only want one row per school. As of now, the DataFrame has multiple rows per school to give metrics across different reporting groups.

To do this, we will construct of matrix where rows are individual schools and columns are 
["RB", "RI", "RA", "RF", "RD", "RP", "RT", "RW", "GM", "GF", "GX", "GZ", "SE", "SD", "SS", "SM", "SF", "SH", "TA"]

These are each of the reporting categories.

In [6]:
def generate_column_index_mapping(columns):
    mapping = {}
    for i,col in enumerate(columns):
        mapping[col] = i
    return mapping
mapping = generate_column_index_mapping(["RB", "RI", "RA", "RF", "RH", "RD", "RP", "RT", "RW", "GM", "GF", "GX", "GZ", "SE", "SD", "SS", "SM", "SF", "SH", "TA"])

In [7]:
all_columns = ["AcademicYear", "AggregateLevel", "CountryCode", "DistrictCode", "SchoolCode", "CountyName",
          "DistrictName", "SchoolName", "CharterYN", "CAR_RB", "CAR_RI", "CAR_RA", "CAR_RF", "CAR_RH", "CAR_RD",
          "CAR_RP", "CAR_RT", "CAR_RW", "CAR_GM", "CAR_GF", "CAR_GX", "CAR_GZ", "CAR_SE", "CAR_SD", "CAR_SS",
          "CAR_SM", "CAR_SF", "CAR_SH", "CAR_TA"]
reporting_category_columns = ["CAR_RB", "CAR_RI", "CAR_RA", "CAR_RF", "CAR_RH", "CAR_RD",
          "CAR_RP", "CAR_RT", "CAR_RW", "CAR_GM", "CAR_GF", "CAR_GX", "CAR_GZ", "CAR_SE", "CAR_SD", "CAR_SS",
          "CAR_SM", "CAR_SF", "CAR_SH", "CAR_TA"]

In [8]:
def colapse_df(df):
    #determine which rows are associated with each school
    school_codes = df["SchoolCode"].unique()
    associated_rows = {}
    for code in school_codes:
        associated_rows[code] = []
    for i in range(df.shape[0]):
        #df["SchoolCode"][df.index[i]] = school code at row i of df
        associated_rows[df["SchoolCode"][df.index[i]]].append(i)
        
    array = np.resize(np.array(all_columns), (1,len(all_columns))) #resize to stack rows with numpy
    for code in school_codes:
        rows = associated_rows[code] #get rows associated with code
        firstrow = df.iloc[rows[0]] #use first row to capture redundant data
        school_data = np.array([[firstrow["AcademicYear"], firstrow["AggregateLevel"], firstrow["CountyCode"], 
                               firstrow["DistrictCode"], firstrow["SchoolCode"], firstrow["CountyName"],
                               firstrow["DistrictName"], firstrow["SchoolName"], firstrow["CharterYN"],
                                       None,None,None,None,None,None,None,None,None,None,None,None,None,
                                       None,None,None,None,None,None,None]])
        #note that all reporting categories data is initialized as None so we can determine what was missing
        for row in rows:
            #write reporting category data to correct column
            school_data[0][9+mapping[df.iloc[row]["ReportingCategory"]]] = df.iloc[row]["ChronicAbsenteeismRate"]
        array = np.append(array,school_data,axis=0) #stack rows
    return pd.DataFrame(array[1:,:], columns=all_columns) #convert to df and return 

In [9]:
new_df1 = colapse_df(df1)
new_df2 = colapse_df(df2)

In [16]:
new_df1.iloc[0]

AcademicYear                                     2016-17
AggregateLevel                                         S
CountryCode                                            1
DistrictCode                                       10017
SchoolCode                                        112607
CountyName                                       Alameda
DistrictName          Alameda County Office of Education
SchoolName        Envision Academy for Arts & Technology
CharterYN                                            All
CAR_RB                                              23.5
CAR_RI                                               NaN
CAR_RA                                               NaN
CAR_RF                                               NaN
CAR_RH                                                23
CAR_RD                                               NaN
CAR_RP                                               NaN
CAR_RT                                               NaN
CAR_RW                         

In [15]:
rows = []
for i in range(df1.shape[0]):
    if df1["SchoolCode"][df1.index[i]] == 112607:
        rows.append(i)
df1.iloc[rows]

,AcademicYear,AggregateLevel,CountyCode,DistrictCode,SchoolCode,CountyName,DistrictName,SchoolName,CharterYN,ReportingCategory,ChronicAbsenteeismEligibleCumula,ChronicAbsenteeismCount,ChronicAbsenteeismRate
53047,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,GF,221.0,58.0,26.2
53056,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,GM,194.0,41.0,21.1
53098,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,RA,NaN,NaN,NaN
53106,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,RB,153.0,36.0,23.5
53115,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,RD,NaN,NaN,NaN
53118,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,RF,NaN,NaN,NaN
53127,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,RH,217.0,50.0,23.0
53136,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,RI,NaN,NaN,NaN
53142,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,RP,NaN,NaN,NaN
53148,2016-17,S,1,10017.0,112607.0,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,All,RT,NaN,NaN,NaN


### Step 3
Write a function that generates a single DataFrame given multiple dataframes from different time periods of the same category. The resultant DataFrame should organize each school's data in chronological order.